In [1]:
#0 Set anything that needs to be changed at the beginning:  $$$ LabelPath and GraphArtifactsFolderName
# You have to restart the kernel each time it seems or sometimes Cell-22 spits an  error!!
# Better clear all output too while you're at it!
# Let's call this: VER2.1, Version-2.1: orders-4, output*0.38mm
# Cases Done: [71,2,37,18,16,88,19,90,11,6,46,39,22,21,95,84,50,47,56,53,58,49,71,79,38,43, ]
import os
# Only need to change here: Look for S-Final---label.nrrd:
LabelPath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-32-Spiromics-51292112/Markups/88-Jensen/S-Final-2-label.nrrd"

GraphArtifactsFolderName = "z:/Q- Figures/T-32Poster/Nate/GraphCreationCodeFiles/Case-32"
if not os.path.exists(GraphArtifactsFolderName):
    os.mkdir(GraphArtifactsFolderName)
verbose = False # Set to True to see a lot more print outputs

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'z:/Q- Figures/T-32Poster/Nate/GraphCreationCodeFiles/Case-32'